<h1>Read the univariant regression dataset files and prepare the data for training, testing, and validation</h1>

In [6]:
import pandas as pd
import numpy as np

#### Path of Univariant regression dataset 
f1 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Regression\UnivariateData\32.csv"

### Devide the data into training, validation, and testing data
df = pd.read_csv(f1, header=None)
train, validate, test = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
trainX, trainY = train.to_numpy()[:,0].reshape((train.shape[0], 1)), train.to_numpy()[:,1].reshape((train.shape[0], 1))
validateX, validateY = validate.to_numpy()[:,0].reshape((validate.shape[0], 1)), validate.to_numpy()[:,1].reshape((validate.shape[0], 1))
testX, testY = test.to_numpy()[:,0].reshape((test.shape[0], 1)), test.to_numpy()[:,1].reshape((test.shape[0], 1))


<h1>Implementation of multilayer feed forward neural network (MLFFNN)</h1> 

In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

### Activation Functions Definitions
class Sigmoid():
    def __call__(self, x, b=1):
        return 1.0/(1.0 + np.exp(-(b*x)))
    def gradient(self, x, b=1):
        return self.__call__(x, b) * (1 - self.__call__(x, b))

class Linear():
    def __call__(self, x, b=1):
        return b*x
    def gradient(self, x, b=1):
        return b

### Multilayer feed forward neural network class
class MLFFNN():
    def __init__(self, n_hidden, n_epoch=1000, learning_rate=0.01, threshold=0.001):
        self.n_hidden = n_hidden
        self.n_epoch = n_epoch
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.hidden_activation = Sigmoid()
        self.output_activation = Linear()

    ### Initialize the weights of neural network
    def initialize_weights(self, X, y):
        n_samples, n_features = X.shape
        _, n_outputs = y.shape
        
        ### For all hidden layers
        pre_num_of_neuron = n_features
        self.weights = {}
        self.w0 = {}
        for i in range(len(self.n_hidden)):
            limit   = 1 / math.sqrt(pre_num_of_neuron)
            self.weights[i]  = np.random.uniform(-limit, limit, (pre_num_of_neuron, self.n_hidden[i]))
            self.w0[i] = np.zeros((1, self.n_hidden[i]))
            pre_num_of_neuron = self.n_hidden[i]
        
        # For output layer
        limit   = 1 / math.sqrt(pre_num_of_neuron)
        self.V  = np.random.uniform(-limit, limit, (self.n_hidden[-1], n_outputs))
        self.v0 = np.zeros((1, n_outputs))

    def train(self, X, y, epoch=True):
        self.initialize_weights(X, y)
        self.errors = []
        ### This conditional block of code is for fixed number of epoch
        if epoch == True:
            ### Run it for n_epoch times
            for i in range(self.n_epoch):
                ### For hidden layer
                inputs = X
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)

                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
        ### This conditional block of code is for fixed threshold of average error
        else:
            error = 10000000
            noOfNoChangeError = 0
            ### Run it until error converges to the threshhold
            while error > self.threshold:
                ### For hidden layer
                inputs = X
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)
                
                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
                ### If there is no change in error
                if noOfNoChangeError >=20:
                    break
                else:
                    if len(self.errors) >= 4:
                        if error == self.errors[-1] and error == self.errors[-2]:
                            noOfNoChangeError += 1
                error = self.errors[-1]
                
    ### Forward Calculation ###
    def forward_calculation(self, inputs):
        ### For hidden layer
        for i in range(len(self.n_hidden)):
            ### Input to neuron
            self.hidden_input[i] = inputs.dot(self.weights[i]) + self.w0[i]
            ### Output of neuron
            self.hidden_output[i] = self.hidden_activation(self.hidden_input[i])
            inputs = self.hidden_output[i]
        ### For output layer
        self.output_layer_input = inputs.dot(self.V) + self.v0
        self.y_pred = self.output_activation(self.output_layer_input)
        return self.y_pred
    
    ### Backpropagation Calculation ###
    def backpropagation_calculation(self, inputs, y):
        ### First for output layer
        ### Gradient w.r.t input of output layer
        grad_wrt_out_l_input = self.loss(y, self.y_pred) * self.output_activation.gradient(self.output_layer_input)
        grad_v = self.hidden_output[len(self.n_hidden)-1].T.dot(grad_wrt_out_l_input)
        grad_v0 = np.sum(grad_wrt_out_l_input, axis=0, keepdims=True)
        ### For hidden layer
        ### Gradient w.r.t input of hidden layer
        next_grad_wrt_hidden_l_input = grad_wrt_out_l_input
        next_weight = self.V
        prev_input = inputs
        grad_w = {}
        grad_w0 = {}
        ### Calculation for multiple hidden layer starting from last to first hidden layer
        for i in reversed(range(len(self.n_hidden))):
            grad_wrt_hidden_l_input = next_grad_wrt_hidden_l_input.dot(next_weight.T) * self.hidden_activation.gradient(self.hidden_input[i])
            ### If hidden layer not connected to input layer
            if i != 0:
                grad_w[i] = self.hidden_output[i-1].T.dot(grad_wrt_hidden_l_input)
            ### when hidden layer connected to input layer
            else:
                grad_w[i] = inputs.T.dot(grad_wrt_hidden_l_input)
            grad_w0[i] = np.sum(grad_wrt_hidden_l_input, axis=0, keepdims=True)
            next_grad_wrt_hidden_l_input = grad_wrt_hidden_l_input
            next_weight = self.weights[i]

        ### Calculaton for weights update ###
        ### Weights update of output layer
        self.V  -= self.learning_rate * grad_v
        self.v0 -= self.learning_rate * grad_v0
        ### Weights update of hidden layers
        for i in range(len(self.n_hidden)):
            self.weights[i]  -= self.learning_rate * grad_w[i]
            self.w0[i] -= self.learning_rate * grad_w0[i]
    
    ### Prediction Function
    def predict(self, X):
        ### Call Forward Calculation ###
        y_pred = self.forward_calculation(X)
        return y_pred
    
    ### Instantaneous error and loss function
    def SquareLoss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2)
    def loss(self, y, y_pred):
        return -(y - y_pred)

### Calculate the accuracy
def accuracy_score(y, y_pred):
    accuracy = np.sum(y == y_pred, axis=0) / len(y)
    return accuracy*100

### Plot of Epoch vs Mean Square Error
def epochVsError_plot(model):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.errors
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Linearly Seperable data using MLFFNN')
    plt.savefig("AvgErrorVsEpoch_MLFFNN_Regression1.png", dpi=600, bbox_inches="tight")
    plt.clf()

### Scatter plot for output of each nueron of hidden layer and output layer plot
def output_nueron_plot(X_train, y_train, z, label='hidden', layer='', nn=''):
    # Creating figure
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    # Add x, y gridlines 
    ax.grid(b = True, color ='grey', linestyle ='-.', linewidth = 0.3, alpha = 0.2) 
    # Creating plot
    sctt = ax.scatter(X_train[:,0], z, s=10, c=z, cmap='Blues')
    ax.set_xlabel('X-axis', fontweight ='bold') 
    ax.set_ylabel('Y-axis (Neuron Output)', fontweight ='bold') 
    # save plot
    plt.savefig("Output_of_{}_layer_{}_Nueron_{}_Regression1.png".format(label, layer, nn), dpi=600, bbox_inches="tight")
    plt.clf()

def modelAndTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    n_item = [i+1 for i in range(len(target))]
    plt.scatter(n_item, model, marker='o', s=5, facecolors='b', edgecolors='b', label='Model Output')
    plt.scatter(n_item, target, marker='s', s=5, facecolors='r', edgecolors='r', label='Target Output')
    plt.legend(bbox_to_anchor=(0.15, 1.2), loc='upper left', ncol=2)
    plt.xlabel('Dataset')
    plt.ylabel('Regression Output')
#     plt.title('Model output and target output of {} dataset'.format(label))
    plt.savefig("modelAndTarget_{}_MLFFNN_Regression1.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()

def modelVsTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.scatter(target, model, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Target Output')
    plt.ylabel('Model Output')
#     plt.title('Model output vs target output of {} dataset'.format(label))
    plt.savefig("modelVsTarget_{}_MLFFNN_Regression1.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()

def MSE(target, model):
    target = np.array(target)
    model = np.array(model)
    mse = np.mean(np.power(target - model, 2))
    return mse

def plot_mse(mse):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    x_data = ['train', 'validation', 'test']
    plt.scatter(x_data, mse, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Type of Dataset')
    plt.ylabel('Mean Square Error')
#     plt.title('Mean squared error (MSE) on training data, validation data and test data')
    plt.savefig("MSE_MLFFNN_Regression1.png", dpi=600, bbox_inches="tight")
    plt.clf()
    
def main():
    ### Call the MLFFNN calss 
    mlffnn = MLFFNN(n_hidden=[3,3], n_epoch=5000, learning_rate=0.0001, threshold=0.001)
    ### Train the MLFFNN
    mlffnn.train(trainX, trainY, epoch=True)
    
    ### Plots for output of each nueron of hidden layer and output layer plot
    for i in range(len(mlffnn.n_hidden)):
        for k in range(mlffnn.hidden_output[i].shape[1]):
            z = mlffnn.hidden_output[i][:,k]
            output_nueron_plot(trainX, trainY, z, label='hidden', layer=i+1, nn=k+1)
    for k in range(mlffnn.y_pred.shape[1]):
        z = mlffnn.y_pred[:,k]
        output_nueron_plot(trainX, trainY, z, label='output', nn=k+1)
            
    ### Prediction for train data
    y_pred_train = mlffnn.predict(trainX)
    y_train = trainY
    ### Prediction for validation data
    y_pred_val = mlffnn.predict(validateX)
    y_val = validateY
    ### Prediction for test data
    y_pred_test = mlffnn.predict(testX)
    y_test = testY
    
    ### Epoch vs error plot
    epochVsError_plot(mlffnn)
    ### Model output and target output plot
    modelAndTarget_plot(y_train, y_pred_train, label='train')
    modelAndTarget_plot(y_test, y_pred_test, label='test')
    modelAndTarget_plot(y_val, y_pred_val, label='validation')
    ### Model output vs target output plot
    modelVsTarget_plot(y_train, y_pred_train, label='train')
    modelVsTarget_plot(y_test, y_pred_test, label='test')
    modelVsTarget_plot(y_val, y_pred_val, label='validation')
    ### For Mean square Error
    mse_train = MSE(y_train, y_pred_train)
    mse_val = MSE(y_val, y_pred_val)
    mse_test = MSE(y_test, y_pred_test)
    plot_mse([mse_train, mse_val, mse_test])

if __name__ == "__main__":
    main()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

Training accuracy 50.22
